# Домашнее задание №5
## Знакомство с языком обработки данных SAS Base
### PROC SQL

_[Укажите в данной ячейке ФИО всех участников вашей группы - от 1 до 3 человек]_

Пучкин Данила Андреевич

### 0. Подготовка файлов для выполнения ДЗ.
Вместе с данным файлом в архиве находятся данные для выполнения ДЗ: 
- наборы данных SAS: 
    Employee_donations, 
    Employee_Payroll,
    product_dim,
    order_fact,
    customer,
    TRNS;

Скопируйте данные в директорию на вашем компьютере, которая связана с виртуальной машиной. Далее всю работу мы будем проводить в этой директории.


In [1]:
libname Orion '/folders/myfolders/HW5/HW5';

SAS Connection established. Subprocess id is 2695



### 1.1 Коррелированный подзапрос 

Выведите информацию (т.е. переменные `Employee_ID`, `Employee_Gender` и `Marital_Status`) о всех сотрудниках, которые суммарно за 4 квартала жертвуют более 0.002 от своей заработной платы. Таблица **orion.Employee_donations** содержит переменные: `Employee_ID`, ежеквартальные пожертвования (переменные `Qtr1`-`Qtr4`), и благотворительные организации (`Recipients`). Таблица **orion.Employee_Payroll** содержит `Employee_ID` и информацию о заработной плате (`Salary`). 

In [2]:
proc print data=orion.Employee_donations (obs=5); run;
proc print data=orion.Employee_Payroll (obs=5); run;

In [3]:
proc sql outobs=5;
    select D.Employee_ID, P.Employee_Gender, P.Marital_Status
    from orion.Employee_donations as D 
         inner join 
         orion.Employee_Payroll as P
         on D.Employee_ID=P.Employee_ID
    where 0.002*P.Salary < (select sum(Qtr1, Qtr2, Qtr3, Qtr4)
                            from orion.Employee_donations E
                            where E.Employee_ID = P.Employee_ID)
    /* ... */
    ; 
quit;

Employee ID,Employee_Gender,Marital_Status
120267,F,M
120680,F,S
120686,F,M
120689,F,S
120753,M,S


### 1.2 Соединение таблиц

Выведете информацию о клиентах компании Orion Star покупающих по каталогу и в интернет, проживающих в США или Австралии, которые приобрели продукцию иностранного производства (то есть ту, которая не была произведена в стране их проживания). Должны отображаться имена клиентов и количество сделанных иностранных покупок (то есть количество заказов, содержащих иностранные продукты).

Отсортируйте резульатат так, чтобы клиенты, у которых наибольшее количество покупок, отображались в верхней части отчета, а клиенты, имеющие одинаковое количество покупок, отображались в алфавитном порядке.

Employee_ID = 99999999 - это фиктивный ID для идентификации покупок по каталогу и в интернет.
Необходимые данные можно найти в перечисленных столбцах следующих таблиц:
-  **product_dim**
     - `Product_ID`
     - `Supplier_Country`
-  **order_fact**
     - `Product_ID`
     - `Employee_ID`
     - `Customer_ID`
-  **customer**
     - `Customer_ID`
     - `Country`

In [4]:
proc print 
    data=orion.product_dim (obs=5); 
    var Product_ID Supplier_Country;
run;
proc print 
    data=orion.order_fact (obs=5); 
    var Customer_ID Employee_ID Order_ID Product_ID;
    where Employee_ID EQ 99999999 and Customer_ID EQ 23;
run;

proc print 
    data = orion.customer (obs=5);
    var Customer_ID Country Customer_Name;
run;

In [5]:
proc sql outobs=5;
    select Distinct OC.Customer_Name, count(distinct OC.Order_ID) as Foreigh
    from orion.product_dim as P 
         inner join 
         (select O.Product_ID, O.Order_ID, O.Customer_ID, C.Customer_Name, C.Country
          from orion.order_fact as O
               inner join
               orion.customer as C
               on O.Customer_ID=C.Customer_ID
          where (O.Employee_ID = 99999999) and (C.Country in ("US", "AU"))
         ) as OC
         on P.Product_ID=OC.Product_ID
    where OC.Country ^= P.Supplier_Country 
    group by OC.Customer_ID 
    order by Foreigh desc, Customer_Name
    /* ... */
    ;
quit;

Customer Name,Foreigh
Candy Kinsey,9
Phenix Hill,6
Cynthia Mccluney,5
Karolina Dokter,4
Sandrina Stephano,4


### 1.3 Операторы работы с множествами
Создайте таблицы А и B для выполнения заданий п. 1.3. Внимательно изучите примеры, выполниет приведенный код.

In [6]:
****************************************************;
* Create input data A and B;
****************************************************;
data work.A;
infile cards;
length x y $5 ;
input x y $ ;
cards;
1 one
2 two
2 two
3 three
;
run;
title 'Table A';
proc print; run;
data work.B;
infile cards;
length x z $5 ;
input x z $ ;
cards;
1 one
2 two
4 four
;
run;
title 'Table B';
proc print; run;
title;

Obs,x,y
1,1,one
2,2,two
3,2,two
4,3,three
Obs,x,z
1,1,one
2,2,two
3,4,four


### 1.3.1  Объединение уникальных строк двух таблиц  (UNION)

Оператор **UNION** объединяет две таблицы. В результирующей таблице находятся все уникальные строки первой и второй таблиц,т.е. каждая строка результирующей таблицы находится либо в первой таблице, либо во второй, либо в двух одновременно. При этом **UNION** не возвращает дубли, т.е. если строка встречается более одного раза, то в итоговой таблице эта строка встретится только один раз.

In [7]:
proc sql;
   title 'A UNION B';
   select * from a
   union
   select * from b;
quit;

x,y
1,one
2,two
3,three
4,four


### 1.3.2 Объединение двух таблиц (UNION ALL)

Вы можете использовать ключевое вслово **ALL**, чтобы оставить дубли в результирующей таблице.

In [8]:
proc sql;
   title 'A UNION ALL B';
   select * from a
   union all
   select * from b;
quit;

x,y
1,one
2,two
2,two
3,three
1,one
2,two
4,four


### 1.3.3 Строки, встречающиеся только в первой таблицы  (EXCEPT)

Оператор **EXCEPT** возвращает строки, которые содержатся в первой таблице, но не содержатся во второй. 

In [9]:
proc sql;
   title 'A EXCEPT B';
   select * from a
   except
   select * from b;
quit;

x,y
3,three


### 1.3.4 Строки, встречающиеся только в первой таблицы  (EXCEPT ALL)

Оператор **EXCEPT** не возвращает дубликаты. Чтобы этого избежать, можно добавить **ALL**.

In [10]:
proc sql;
   title 'A EXCEPT ALL B';
   select * from a
   except all
   select * from b;
quit;

x,y
2,two
3,three


Подумайте, почему появилась строка {2, two}!

### 1.3.5 Персечение двух таблиц (INTERSECT)

Оператор **INTERSECT** возвращает строки, которые одновременно находятся в первой и второй таблицах (в пересечении таблиц).

In [11]:
proc sql;
   title 'A INTERSECT B';
   select * from a
   intersect
   select * from b;
quit;

x,y
1,one
2,two


### 1.3.6  Пересечение двух таблиц (INTERSECT ALL)

Результатом оператора **INTERSECT ALL** является таблица, содержащая строки первой и второй таблицы, "объединенные" один к одному.

In [12]:
proc sql;
   title 'A INTERSECT B';
   select * from a
   intersect all
   select * from b;
quit;

x,y
1,one
2,two


### 1.3.7 Конкатенация таблиц (OUTER UNION)

Оператор **OUTER UNION**  просто объединяет таблицы. 

In [13]:
proc sql;
   title 'A OUTER UNION B';
   select * from a
   outer union
   select * from b;
quit;

x,y,x,z
1,one,,
2,two,,
2,two,,
3,three,,
,,1,one
,,2,two
,,4,four


### 1.3.8 Конкатенация таблиц (OUTER UNION CORR)

Оператор **OUTER UNION**  не совмещает столбцы (с одинаковыми именами). Чтобы этого избежать, можно добавить ключевое слово **CORRESPONDING** (или, сокращенно, **CORR**).

In [14]:
proc sql;
   title 'A OUTER UNION CORR B';
   select * from a
   outer union corr
   select * from b;
quit;

x,y,z
1,one,
2,two,
2,two,
3,three,
1,,one
2,,two
4,,four


### 1.3.9 Уникальные строки из таблиц (т.е. вне пересечения таблиц) 


В **PROC SQL** нет оператора, который возвращает уникальные строки из первой и второй таблиц за исключением строк из пересечения. Вот один из вариантов, с помощью которого можно получить описанную таблицу:

`(query1 except query2)
union
(query2 except query1)`

In [15]:
proc sql;
   title 'A EXCLUSIVE UNION B';
   (select * from a
         except
         select * from b)
   union
   (select * from b
         except
         select * from a);
quit;

x,y
3,three
4,four


Первый оператор **EXCEPT**  возвращает одну уникальную строку только из первой таблицы (таблица А). Второй оператор **EXCEPT**  возвращает одну уникальную строку только из второй таблицы (таблица В). Оператор **UNION** объединяет оба результата.

### 1.4  Remerging
У нас имеется таблица с покупками, совершенными клиентами продуктового магазина за один месяц - TRNS. Нужно рассчитать долю трат, для каждого покупателя на каждую категорию товаров. И вывести для каждого покупателя категорию, которую он предпочитает - т .е. тратит на нее максимум денег.  Кто-то приходит в магазин в основном за сигаретами и тратит на них 95% всех потраченных им денег, а кто-то покупает только продовольственные товары -  траты на них составляют 100%.

#### 1.4.1  Пример 
Следующий пример должен помочь выполнить задание: при вычислении колонки Percentage в знаменателе используется агрегатная функция SUM(), при этом нет выражения group by, значит данный запрос выполнится в два шага: (1) сначала вычислится значение SUM() по всей таблице (если бы был group by, то внутри групп), (2) затем будет использовано вычисленное значение, чтобы рассчитать переменную Percentage. Для этого понадобится два прохода по данным, с соединением результатов (remerging) первого прохода обратно к таблице для получения финального результата на втором проходе.   

In [123]:
/* ПРИМЕР */
proc sql noexec;
    select 
        Name /*название страны*/, 
        Population /*население, чел.*/,
        (Population / sum(Population) * 100) as Percentage /* доля, которую составляет население страны от мирового */
      from countries
      order by Percentage desc;
quit;

In [124]:
/*пример вычисляет немного не то - процент конкретной покупки от суммарных трат по всем пользователям*/
proc sql outobs=15;
    select 
        CID, 
        CAT,
        SumRub,
        (SumRUB / sum(SumRUB) * 100) as Percentage /* доля, которую составляет население страны от мирового */
      from orion.trns
      order by CID, Percentage desc;
quit;

CID,CAT,SumRUB,Percentage
00250669,FOOD,796,0.006724
00250669,FOOD,327,0.002762
00250669,FOOD,220,0.001858
00250669,NONFOOD,219,0.00185
00250669,FOOD,211,0.001782
00250669,FOOD,164,0.001385
00250669,NONFOOD,121,0.001022
00250669,FOOD,113,0.000955
00250669,FOOD,100,0.000845
00250669,FOOD,90,0.00076


#### 1.4.2  оператор Describe 

Посмотрим на структуру таблицы TRNS: оператор `describe table` выводит в лог оператор `create table`, с помощью которого создавалась таблица.

In [125]:
title;

proc sql;
    create table TRNS as
    select * from orion.trns;
run;

proc sql;
    describe table TRNS;
quit;

proc print data = orion.trns (obs=5); run;

Obs,TRID,CID,Date,SumRUB,PID,CAT
1,000336660,00286578,01/07/2017,15,0112060,NONFOOD
2,000336660,00286578,01/07/2017,40.42,0005451,FOOD
3,000336660,00286578,01/07/2017,45,0122407,FOOD
4,000336660,00286578,01/07/2017,75,0118683,FOOD
5,000336660,00286578,01/07/2017,75,0123303,COOKERY


#### 1.4.3  Колонки в таблице TRNS:
 - TRID - идентификатор транзакции, т.е. номер чека (в одном чеке может быть куплено несколько товаров),
 - CID - идентификатор клиента, т.е. того, кто совершил покупку,
 - Date - дата покупки,
 - PID - идентификатор купленного товара,
 - CAT - категория товара (кулинария, продовольсвенные товары, непродовольственные товары, табак).
 - SumRUB - стоимость товара ().
   
#### 1.4.4 Примерный план решения:
 1. вычислить сумму, потраченную _каждым клиентом_ на _каждую категорию_ - `group by` 
 2. вычислить долю трат: отношение трат _клиента_ в категории к суммарным тратам клиента - `sum_cat/sum(sum_cat)` 
 3. оставить только строки по категориям с максимальной долей - ``having``
 4. вы можете использовать подзапросы или сделать несколько вызовов proc sql; 
 
 **Внимание: результатом должна быть _таблица_ share_cat**

In [126]:
proc sql;
    create table share_cat as
    select CID, CAT, 100*SumTotal/sum(SumTotal) as Percentage
    from (select CID, CAT, sum(SumRUB) as SumTotal
          from TRNS
          group by CID, CAT)
    group by CID
    having SumTotal = max(SumTotal)
/* ... */
    ;
run;

proc print data= share_cat (obs=15); run;

Obs,CID,CAT,Percentage
1,00250669,FOOD,85.531
2,00250673,FOOD,100.000
3,00250679,FOOD,48.639
4,00250685,FOOD,73.956
5,00250691,FOOD,50.507
6,00250698,COOKERY,100.000
7,00250701,FOOD,76.402
8,00250710,FOOD,43.129
9,00250718,FOOD,99.411
10,00250719,COOKERY,99.326
